In [19]:
import os
from PIL import Image
import numpy as np
import cv2

SOURCE_DIR = "D:\dataset\Samsung\IDK"
TARGET_DIR_SAP = "D:\dataset\Samsung\salted papaer noise"
TARGET_DIR_GAU = "D:\dataset\Samsung\gaussian noise"

os.makedirs(TARGET_DIR_SAP, exist_ok=True)
os.makedirs(TARGET_DIR_GAU, exist_ok=True)


GAUSSIAN_STD_DEV = 0.08  # רמת סטיית התקן עבור רעש גאוסיאני
SALT_PEPPER_DENSITY = 0.04 # צפיפות הרעש עבור מלח ופלפל (לדוגמה: 4%)
PATCH_SIZE = 50   # גודל הטלאי (50x50 פיקסלים)

In [15]:
def add_gaussian_noise(img: np.ndarray, noise_level: float = 0.05) -> np.ndarray:

    MAX_PIXEL_VALUE = 255.0
    mean = 0
    std_dev = noise_level * MAX_PIXEL_VALUE

    if std_dev <= 0:
        return img.copy()

    noise = np.random.normal(mean, std_dev, img.shape)

    noisy_image_float = img.astype(np.float32) + noise

    noisy_image = cv2.convertScaleAbs(noisy_image_float)

    return noisy_image

In [16]:
def add_salt_and_pepper_noise(img: np.ndarray, density: float = 0.04) -> np.ndarray:

    noisy_img = img.copy()

    H, W = img.shape[:2]
    total_pixels_2d = H * W
    num_corrupt_pixels = int(total_pixels_2d * density)

    if num_corrupt_pixels == 0:
        return noisy_img

    coords_H = np.random.randint(0, H, size=num_corrupt_pixels)
    coords_W = np.random.randint(0, W, size=num_corrupt_pixels)

    num_salt = num_corrupt_pixels // 2

    noisy_img[coords_H[:num_salt], coords_W[:num_salt]] = 255

    noisy_img[coords_H[num_salt:], coords_W[num_salt:]] = 0

    return noisy_img

In [21]:

processed_patches_count = 0

for img_name in os.listdir(SOURCE_DIR):
    img_path = os.path.join(SOURCE_DIR, img_name)

    if not os.path.isfile(img_path) or not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    try:
        img = Image.open(img_path).convert("RGB")
        img_width, img_height = img.size

        base_name = os.path.splitext(img_name)[0]
        patch_id = 0

        for top in range(0, img_height, PATCH_SIZE):
            for left in range(0, img_width, PATCH_SIZE):

                right = min(left + PATCH_SIZE, img_width)
                bottom = min(top + PATCH_SIZE, img_height)

                if (right - left) != PATCH_SIZE or (bottom - top) != PATCH_SIZE:
                    continue

                patch_pil = img.crop((left, top, right, bottom))

                patch_np = np.array(patch_pil)

                noisy_patch_np = add_gaussian_noise(patch_np, noise_level=GAUSSIAN_STD_DEV)

                noisy_patch_pil = Image.fromarray(noisy_patch_np)

                noise_int = int(GAUSSIAN_STD_DEV * 100)
                patch_filename = f"{base_name}_noise{noise_int}_patch{patch_id}.png"

                noisy_patch_pil.save(os.path.join(TARGET_DIR_GAU, patch_filename))

                patch_id += 1
                processed_patches_count += 1

    except Exception as e:
        print(f"Skipping {img_name}, error: {e}")

print(f"Finished processing. Total patches saved to disk: {processed_patches_count} in {TARGET_DIR_GAU}")

Finished processing. Total patches saved to disk: 96152 in D:\dataset\Samsung\gaussian noise


In [18]:

processed_patches_count = 0

for img_name in os.listdir(SOURCE_DIR):
    img_path = os.path.join(SOURCE_DIR, img_name)

    if not os.path.isfile(img_path) or not img_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
        continue

    try:
        img = Image.open(img_path).convert("RGB")
        img_width, img_height = img.size

        base_name = os.path.splitext(img_name)[0]
        patch_id = 0

        for top in range(0, img_height, PATCH_SIZE):
            for left in range(0, img_width, PATCH_SIZE):

                right = min(left + PATCH_SIZE, img_width)
                bottom = min(top + PATCH_SIZE, img_height)

                if (right - left) != PATCH_SIZE or (bottom - top) != PATCH_SIZE:
                    continue

                patch_pil = img.crop((left, top, right, bottom))

                patch_np = np.array(patch_pil)

                noisy_patch_np = add_salt_and_pepper_noise(patch_np, density=SALT_PEPPER_DENSITY)

                noisy_patch_pil = Image.fromarray(noisy_patch_np)

                noise_int = int(SALT_PEPPER_DENSITY * 100)
                patch_filename = f"{base_name}_noise{noise_int}_patch{patch_id}.png"

                noisy_patch_pil.save(os.path.join(TARGET_DIR_SAP, patch_filename))

                patch_id += 1
                processed_patches_count += 1

    except Exception as e:
        print(f"Skipping {img_name}, error: {e}")

print(f"Finished processing. Total patches saved to disk: {processed_patches_count} in {TARGET_DIR_SAP}")

Finished processing. Total patches saved to disk: 96152 in D:\dataset\Sumsung\salted papaer noise
